In [53]:
import antlr4
from antlr4 import *
from BSLLexer import BSLLexer
from BSLParser import BSLParser
from BSLParserVisitor import BSLParserVisitor

input_stream = FileStream('ObjectModule.bsl', encoding='utf-8')
lexer = BSLLexer(input_stream)
stream = CommonTokenStream(lexer)
parser = BSLParser(stream)
tree = parser.file_()  # Пример именования стартового правила

In [55]:
from BSLMethodDescriptionLexer import BSLMethodDescriptionLexer
from BSLMethodDescriptionParser import BSLMethodDescriptionParser
from BSLMethodDescriptionParserVisitor import BSLMethodDescriptionParserVisitor
from antlr4 import InputStream

In [56]:
class MethodVisitor(BSLMethodDescriptionParserVisitor):
    def __init__(self):
        self.parse_dict={
            
        }

    def visitDeprecate(self, ctx):
        self.parse_dict.update({'deprecate': ctx.getText()})
        return self.visitChildren(ctx)

    def visitDescriptionBlock(self, ctx):
        self.parse_dict.update({'descriptionBlock': ctx.getText()})
    
    def visitParameters(self, ctx):
        self.parse_dict.update({'parameters': ctx.getText()})
    
    def visitCallOptions(self, ctx):
        self.parse_dict.update({'callOptions': ctx.getText()})


In [57]:
def parse_comment(comment):
    string_stream = InputStream(comment)
    method_lexer = BSLMethodDescriptionLexer(string_stream)
    method_stream = CommonTokenStream(method_lexer)
    method_parser = BSLMethodDescriptionParser(method_stream)
    tree = method_parser.methodDescription()  # Пример именования стартового правила

    visitor = MethodVisitor()
    visitor.visit(tree)

    return visitor.parse_dict

In [58]:
parse_comment('//Hello, world \n// Параметры:\n // Устарела')

{'descriptionBlock': '//Hello, world \n',
 'parameters': '// Параметры:\n',
 'deprecate': ' // Устарела'}

In [59]:
print('// Параметры:\n// Устарела')

// Параметры:
// Устарела


In [70]:
def get_comments_above(ctx):
    met_non_empty_flag = False
    res_str = ""
    token_list = [item.text for item in [token for token in stream.getHiddenTokensToLeft(ctx.start.tokenIndex)]]
    for token in reversed(token_list):
        if '\n' in token:
            if met_non_empty_flag:
                break
        else:
            met_non_empty_flag = True
            res_str = ('\n'.join((res_str, token)) if res_str != '' else token)

    
    return res_str, parse_comment(res_str)

In [71]:

class MyVisitor(BSLParserVisitor):
    def __init__(self):
        self.codeflow = list()

    def visitFunction(self, ctx):
        comm = get_comments_above(ctx)
        self.codeflow.append((ctx.getText(), comm[0], comm[1]))
        return self.visitChildren(ctx)
    
    def visitProcedure(self, ctx):
        comm = get_comments_above(ctx)
        self.codeflow.append((ctx.getText(), comm[0], comm[1]))
        return self.visitChildren(ctx)

In [72]:
visitor = MyVisitor()

In [73]:
visitor.visit(tree)

In [74]:
visitor.codeflow

[('ФункцияОписаниеПлагина(ВозможныеТипыПлагинов)ЭкспортРезультат=НовыйСтруктура;Результат.Вставить("Тип",ВозможныеТипыПлагинов.Загрузчик);Результат.Вставить("Идентификатор",Метаданные().Имя);Результат.Вставить("Представление","Загрузить тесты из файлов");ВозвратНовыйФиксированнаяСтруктура(Результат);КонецФункции',
  '// { Plugin interface',
  {'descriptionBlock': '// { Plugin interface'}),
 ('ПроцедураИнициализация(КонтекстЯдраПараметр)ЭкспортКонецПроцедуры', '', {}),
 ('#ЕслиТолстыйКлиентОбычноеПриложениеТогдаФункцияВыбратьПутьИнтерактивно(КонтекстЯдра,ТекущийПуть="")ЭкспортДиалогВыбораТеста=НовыйДиалогВыбораФайла(РежимДиалогаВыбораФайла.Открытие);ДиалогВыбораТеста.Фильтр="Обработка-тест (*.epf)|*.epf|Отчет-тест (*.erf)|*.erf|Все файлы|*";ДиалогВыбораТеста.МножественныйВыбор=Истина;ДиалогВыбораТеста.ПроверятьСуществованиеФайла=Истина;ДиалогВыбораТеста.ПолноеИмяФайла=ТекущийПуть;Результат=НовыйТекстовыйДокумент;ЕслиДиалогВыбораТеста.Выбрать()ТогдаДлякаждогоПолноеИмяФайлаИзДиалогВыбораТ

In [75]:
import pandas as pd

df = pd.DataFrame(visitor.codeflow, columns=['code', 'comment', 'dict'])

In [76]:
df

,code,comment,dict
0,ФункцияОписаниеПлагина(ВозможныеТипыПлагинов)Э...,// { Plugin interface,{'descriptionBlock': '// { Plugin interface'}
1,ПроцедураИнициализация(КонтекстЯдраПараметр)Эк...,,{}
2,#ЕслиТолстыйКлиентОбычноеПриложениеТогдаФункци...,// { Loader interface,{'descriptionBlock': '// { Loader interface'}
3,"#КонецЕслиФункцияЗагрузить(КонтекстЯдра,Путь)Э...",,{}
4,"ФункцияПолучитьКонтекстПоПути(КонтекстЯдра,Пут...",,{}
5,ФункцияПолучитьКонтекстОбработки(ФайлОбработки...,// } Loader interface,{'descriptionBlock': '// } Loader interface'}
6,ПроцедураПроверитьКорректностьФайла(Файл)ЕслиН...,,{}
7,"ФункцияЗагрузитьФайл(ПостроительДереваТестов,Ф...",,{}
8,ФункцияЗагрузитьТестыВНовомФормате(Построитель...,,{}
9,ФункцияЭтоНовыйФорматОбработки(ЗначТекстОшибки...,,{}
